In [ ]:
from asim.dataset.scene.scene_builder import ArrowSceneBuilder
from asim.dataset.scene.scene_filter import SceneFilter
from nuplan.planning.utils.multithreading.worker_sequential import Sequential


In [ ]:
[
    "796266a84fd65c71",
    "1ef8b2f08cd65f9a",
    "8922138735b45195",
    "d851006613b85e7b",
    "38ef797d14445e44",
    "a317258b9849542f",
    "83ca370d430e59bc",
    "3159261edbf95519",
    "d3d80c3089905994",
    "8d3b06d2311955cb",
    "1d33208c85c95470",
    "e333523147ba5cc9",
    "94fa6640d9f25c43",
    "0eb46626e2345bae",
    "c20a47af332e58a0",
    "c4153e20ef9f5429",
    "51dc7988d78d545f",
    "7348ea10ac0d58fd",
    "32abded47532514d",
    "4c8bd4c5b1015b82",
    "24334adc21055f43",
    "c1afafec7c1f55c6",
    "61af5c7ad0a75a17",
    "4dc74226975f5060",
    "f3b9c29ceb205216",
    "6fd1090347e05103",
    "8564bae63ebf5e42",
    "4656fe861aed509e",
    "f4aa5f3bf2095608",
    "5dc7c5ab6ad450cb",
    "852e81b1ce2d5ce0",
    "9634e64f32b0584d",
    "765a3a60360950d8",
    "d726695b080d5662",
    "646d3f76770a5159",
    "cc0ba43b91395054",
    "5a30039504185834",
    "396ced8ecffe5d92",
    "2583343f0e095b76",
    "940480c560a459dd",
    "c106ea1d18fa5b46",
    "8a1f92741cf25e94",
    "1e1d33afe06a5dc9",
    "cd1295e8085d5f8a",
    "6c68b2a6d8cc5ea9",
    "be3c5bf59f675e00",
    "053be6c85efa5e17",
    "14756446a3a65d51",
    "f8b10bcaa98a53e8",
    "d72ab3f1878c51a0",
    "0ee073448a2657dc",
    "5726da45814a5649",
    "f8aaded37e735549",
    "c7f921589ed956f3",
    "3852b5feb1215add",
    "fc931d0c156057b5",
    "a8d21fc93a3755c6",
    "3ee7077fadab55aa",
    "5845b5db282858b9",
    "36908fc5872d538d",
    "a075179737ad5140",
    "754820401e085625",
    "469cb3e6d6055f87",
    "230150f0e8ff5d4d",
    "7a8c727978005f14",
    "50038631e81b58c8",
    "b7ca09eed06d5090",
    "780103da783e597b",
    "0c72e0f43f9e5dc5",
    "68ad7730c5885243",
    "2cdec17051d25bb4",
    "fe17bdf412f755e4",
    "53632970c56a5207",
    "6ba4abec7990550c",
    "023028c423715317",
    "963ff52c54685dd3",
    "fdef7ee7067859d3",
    "79d99553e8c15302",
    "e3eaafd948b15069",
    "123dfe0afe85550f",
    "956bfd38d4e75fea",
    "c979bfba27385ba1",
    "74e40b83c2e55f8d",
    "2f6a2c0802b75922",
    "a190fbdd675052e7",
    "4c59ff2f17cd5b12",
    "adb469e65ae256aa",
    "00273245ce175544",
    "49ce38623a9d56fc",
    "e00f8ae7c6fd548e",
    "c525191820d65766",
    "2e190d8d1d435577",
    "2e8e6e5238495570",
    "6629f76b367b5305",
    "de8590ad0fc15d29",
    "f520d009788d5346",
    "e50e1bb8fce354b6",
    "da55b0a04d465781",
    "652ae5b39e575205",
    "2d767bf67a655fb6",
]

In [ ]:
import os, psutil
import numpy as np

def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Memory usage: {memory_info.rss / 1024 ** 2:.2f} MB")


num_scenes = 100
split = "nuplan_mini_test"

# log_names = ["2021.06.07.12.54.00_veh-35_01843_02314"]
scene_tokens = None
# scene_tokens = ["1b48bd8c60b3790a"]
log_names = None

scene_filter = SceneFilter(split_names=[split], log_names=log_names, scene_tokens=scene_tokens, duration_s=8.1, history_s=1.0)
scene_builder = ArrowSceneBuilder("/home/daniel/asim_workspace/data")
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

np.random.seed(42)
scenes = list(np.random.choice(scenes, size=num_scenes, replace=False))

# scenes = [scene for scene in scenes if scene.token =="be74699d84ec5662"]

experiment_name = None
print([scene.token for scene in scenes])


In [ ]:
import trace
import traceback
from tqdm import tqdm
from asim.dataset.arrow.conversion import BoxDetectionWrapper, DetectionType
from asim.simulation.gym.demo_gym_env import DemoGymEnv
from asim.simulation.observation.agents_observation import _filter_agents_by_type

from typing import List

from asim.simulation.metrics.sim_agents.sim_agents import get_sim_agents_metrics

from asim.dataset.arrow.conversion import BoxDetection, DetectionType
from asim.dataset.recording.detection.detection import BoxDetectionWrapper

import time

import pandas as pd

action = [1.0, 0.1]  # Placeholder action, replace with actual action logic
env = DemoGymEnv(scenes)

results = []

for scene in tqdm(scenes):
    try:

        agent_rollouts = []

        map_api, ego_state, detection_observation, current_scene = env.reset(scene)
        initial_ego_state = ego_state
        agent_rollouts.append(detection_observation.box_detections)

        result = {}
        result["token"] = scene.token
        for i in range(150):
            ego_state, detection_observation, end = env.step(action)
            agent_rollouts.append(detection_observation.box_detections)
            if end:
                break
        result.update(get_sim_agents_metrics(current_scene, agent_rollouts))
        results.append(result)
    except Exception as e:
        print(current_scene.token)
        traceback.print_exc()
        continue



df = pd.DataFrame(results)
avg_row = df.drop(columns=['token']).mean(numeric_only=True)
avg_row['token'] = 'average'
df = pd.concat([df, pd.DataFrame([avg_row])], ignore_index=True)
df.to_csv("test.csv")
